## 🧮 Numerical Integration: Midpoint, Trapezoidal and Simpson

We compute integral using three rules:
- **Midpoint**
- **Trapezoidal**
- **Simpson’s rule**

For each method and each integral, we compute the **absolute error** using  
$ n = 2^k + 1 $ function evaluations, for $ k = 1, 2, \dots, 12 $.

> Based on exercise from *Guidelines for Numerical Codes* (Langtangen)

## 📐 Problem 1: $$ \int_0^\pi \sin(x) \, dx = 2 $$


We numerically evaluate the integral of $ \sin(x) $ from $ 0 $) to $ \pi $,  
whose exact value is **2**.

This function is:
- **Smooth** (infinitely differentiable)
- **Symmetric** about $ x = \pi/2 $
- **Ideal** for testing numerical integration methods

### 🧮 Define the Integrand Function

We define $ f(x) = \sin(x) $ as a standalone function with a docstring.  
This makes the code readable and reusable.

In [1]:
import math

def f(x):
    """Integrand: f(x) = sin(x). Exact integral from 0 to π is 2."""
    return math.sin(x)

# Integration limits and exact value
a, b = 0.0, math.pi
exact = 2.0

### 📊 Compute Numerical Errors

For each $ n $, we:
1. Approximate the integral using all three methods
2. Compute the **absolute error**: $ |\text{approx} - 2| $

Results are stored for table and analysis.

### 🔌 Import General Integration Algorithms

We use a **reusable module** `integration.py` that implements:
- `midpoint(f, a, b, n)`
- `trapezoidal(f, a, b, n)`
- `simpson(f, a, b, n)`

Each method uses **exactly `n` function evaluations**, where  
\( n = 2^k + 1 \) (always odd, suitable for Simpson’s rule).

In [2]:
from integration import midpoint, trapezoidal, simpson

# n = number of function evaluations = 2^k + 1
k_values = list(range(1, 13))          # k = 1 to 12
n_values = [2**k + 1 for k in k_values]  # n = 3, 5, 9, ..., 4097

In [3]:
# Store errors
errors = {'mid': [], 'trap': [], 'simp': []}

for n in n_values:
    err_mid  = abs(midpoint(f, a, b, n) - exact)
    err_trap = abs(trapezoidal(f, a, b, n) - exact)
    err_simp = abs(simpson(f, a, b, n) - exact)
    
    errors['mid'].append(err_mid)
    errors['trap'].append(err_trap)
    errors['simp'].append(err_simp)

ValueError: Simpson's rule requires n >= 3 and odd (n points)

### 📋 Error Table: $ n = 2^k + 1 $

> 💡 **Simpson’s rule converges extremely fast** for smooth periodic-like functions.  
> You’ll see its error drop to **machine precision** (~1e-16) by $ n = 17 $!

In [ ]:
print(f"{'k':>2} {'n':>6} {'Midpoint':>12} {'Trapezoidal':>12} {'Simpson':>12}")
print("-" * 52)
for i, k in enumerate(k_values):
    n = n_values[i]
    print(f"{k:2d} {n:6d} {errors['mid'][i]:12.2e} "
          f"{errors['trap'][i]:12.2e} {errors['simp'][i]:12.2e}")

## 🔍 What Do We Learn?

- ✅ All methods converge as $ n $ increases.
- 🚀 **Simpson’s rule** achieves **near-zero error** with very few points (thanks to 4th-order accuracy).
- ↔️ **Midpoint and Trapezoidal** are both 2nd-order, but Midpoint is slightly more accurate here due to symmetry.
- 🧠 This problem is **ideal**: smooth, no singularites, bounded domain.

> This illustrates why **Simpson’s rule is preferred** when the integrand is smooth and evaluations are cheap.

### 📈 Convergence Plot (Log-Log)

The slope of each line indicates the **order of convergence**:
- Slope ≈ $-2$ → 2nd order (Midpoint, Trapezoidal)
- Slope ≈ $-4$ → 4th order (Simpson)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(6, 4))
plt.loglog(n_values, errors['mid'],  'o-', label='Midpoint')
plt.loglog(n_values, errors['trap'], 's-', label='Trapezoidal')
plt.loglog(n_values, errors['simp'], '^-', label='Simpson')

# Reference lines for O(n⁻²) and O(n⁻⁴)
n_ref = np.array(n_values, dtype=float)
C2 = errors['mid'][0] * n_values[0]**2
C4 = errors['simp'][2] * n_values[2]**4  # use a later point for stability
plt.loglog(n_ref, C2 * n_ref**(-2), '--', color='gray', linewidth=1, label=r'$\mathcal{O}(n^{-2})$')
plt.loglog(n_ref, C4 * n_ref**(-4), ':', color='black', linewidth=1, label=r'$\mathcal{O}(n^{-4})$')

plt.xlabel('n (function evaluations)')
plt.ylabel('Absolute error')
plt.title(r'Convergence for $\int_0^\pi \sin(x)\,dx$')
plt.legend()
plt.grid(True, which='both', ls='--', alpha=0.6)
plt.tight_layout()
plt.show()